In [25]:
import pandas as pd
import datetime
click = pd.read_csv('click_sep.csv', low_memory=False)
buy = pd.read_csv('buy_sep.csv',low_memory=False)

In [26]:
buy['TimeStamp'] = pd.to_datetime(buy['TimeStamp'])

In [27]:
click['TimeStamp'] = pd.to_datetime(click['TimeStamp'])

In [28]:
# feature 1 
# In each click session, how many items were clicked 
feature1 = click.groupby('SessionID')['ItemID'].count().reset_index(name="item_cnt_per_ses") 
feature1.head()

,SessionID,item_cnt_per_ses
0,9194351,5
1,9194429,2
2,9194513,3
3,9194692,1
4,9194824,1


In [29]:
# feature 2 
# In which weekday did the click activity happened
click['click_weekday'] = pd.DatetimeIndex(click['TimeStamp']).day_name()
feature2 = click[['SessionID','click_weekday']].drop_duplicates()
feature2.head()

,SessionID,click_weekday
0,9293568,Monday
1,9293653,Monday
6,9293662,Monday
8,9293704,Monday
10,9293797,Monday


In [30]:
# feature 3 - average number of clicks per item for each session
avg_clicks = round(clicks_by_items.groupby('SessionID')['item_cnt_in_ses'].mean(),2).reset_index(name="avg_clicks_items")
feature3 = avg_clicks
feature3.head()

,SessionID,avg_clicks_items
0,9194351,5
1,9194429,2
2,9194513,3
3,9194692,1
4,9194824,1


In [31]:
# feature 4
# Clicks per four quaters a day
click['buy_hour'] = pd.DatetimeIndex(click['TimeStamp']).hour
feature4 = click[['SessionID','buy_hour']].drop_duplicates()

click.loc[(click['buy_hour'] <= 6), 'Time_bucket'] = '1' 
click.loc[(click['buy_hour'] <= 12) & (click['buy_hour'] > 6), 'Time_bucket'] = '2'
click.loc[(click['buy_hour'] <= 18) & (click['buy_hour'] > 12), 'Time_bucket'] = '3' 
click.loc[(click['buy_hour'] <= 24) & (click['buy_hour'] > 18), 'Time_bucket'] = '4'

feature4 = click[['SessionID','Time_bucket']].drop_duplicates()
feature4.head()

,SessionID,Time_bucket
0,9293568,3
1,9293653,2
6,9293662,1
7,9293662,2
8,9293704,3


In [32]:
# feature 5
#Top 1 Clicked Category in each Click Session
top_cat = click.groupby(['SessionID','Category'],as_index=False)['TimeStamp'].count()
feature5 = top_cat.groupby('SessionID').apply(lambda x: x.Category[x.TimeStamp.idxmax()]).reset_index()
feature5.columns = ['SessionID','top_click_cat']
feature5.head()

,SessionID,top_click_cat
0,9194351,S
1,9194429,S
2,9194513,S
3,9194692,S
4,9194824,1


In [33]:
# feature 6
#In which hour of day the click activity happened.   
click['click_hour'] = pd.DatetimeIndex(click['TimeStamp']).hour
feature6 = click[['SessionID','click_hour']].drop_duplicates()
feature6.head()

,SessionID,click_hour
0,9293568,18
1,9293653,10
6,9293662,6
7,9293662,7
8,9293704,18


In [34]:
# feature 7
# Session length
last_timestamp = click.groupby('SessionID')['TimeStamp'].max().reset_index(name="max")
first_timestamp = click.groupby('SessionID')['TimeStamp'].min().reset_index(name="min")

length = pd.merge(first_timestamp, last_timestamp, how='left', on='SessionID')
length['max'] = pd.to_datetime(length['max'])
length['min'] = pd.to_datetime(length['min'])
length['session_length'] = (length['max']-length['min']).astype('timedelta64[s]')
session_length = length.iloc[:,[0,3]]
feature7 = session_length

feature7.head()

,SessionID,session_length
0,9194351,320.0
1,9194429,60.0
2,9194513,883.0
3,9194692,0.0
4,9194824,0.0


In [35]:
# feature 8
# How many different items were clicked
click['diff_item']=click.groupby('SessionID')['ItemID'].transform('nunique')
feature8=click[['SessionID','diff_item']].drop_duplicates()
feature8.head()

,SessionID,diff_item
0,9293568,1
1,9293653,4
6,9293662,1
8,9293704,1
10,9293797,4


In [36]:
# feature 9

clicks_by_items = click.groupby(['SessionID'])['ItemID'].count().reset_index(name="item_cnt_in_ses")

feature9 = clicks_by_items
feature9.head()

,SessionID,item_cnt_in_ses
0,9194351,5
1,9194429,2
2,9194513,3
3,9194692,1
4,9194824,1


In [37]:
# feature 10
# How many different categories were clicked
click['diff_cat']=click.groupby('SessionID')['Category'].transform('nunique')
feature10=click[['SessionID','diff_cat']].drop_duplicates()
feature10.head()

,SessionID,diff_cat
0,9293568,1
1,9293653,2
6,9293662,1
8,9293704,1
10,9293797,1


In [38]:
#Feature 11 - Max number of clicks on one item per session. 
# The undelying logic is that if a user repeatedly clicks an item, 
# that means the user is coming back to the same product duirng research and the chance of buying 
# the product increases compared t alternatives.

max_clicks = clicks_by_items.groupby('SessionID')['item_cnt_in_ses'].max().reset_index(name = " max_clicks_items_in_ses")

feature11 = max_clicks
feature11.head()

,SessionID,max_clicks_items_in_ses
0,9194351,5
1,9194429,2
2,9194513,3
3,9194692,1
4,9194824,1


In [39]:
#Predictive Target
#1 if there has been a purchase in that session, 0 if not
df = click.merge(buy[['SessionID','Quantity']], how = 'left', on = 'SessionID').drop_duplicates()
df["pred_target"]=0
df.loc[df["Quantity"]>0, "pred_target"]=1
Pred_Target=df[["SessionID", "pred_target"]].drop_duplicates()
Pred_Target.head()

,SessionID,pred_target
0,9293568,0
1,9293653,0
6,9293662,1
8,9293704,1
12,9293797,1


In [40]:
ABT = feature1.merge(feature2, how='outer', on = ['SessionID']).merge(
    feature3, how='outer', on = ['SessionID']).merge(
    feature4, how='outer', on = ['SessionID']).merge(
    feature5, how='outer', on = ['SessionID']).merge(
    feature6, how='outer', on = ['SessionID']).merge(
    feature7, how='outer', on = ['SessionID']).merge(
    feature8, how='outer', on = ['SessionID']).merge(
    feature9, how='outer', on = ['SessionID']).merge(
    feature10, how='outer', on = ['SessionID']).merge(
    feature11, how='outer', on = ['SessionID']).merge(Pred_Target, how='outer', on = ['SessionID'])

In [41]:
ABT.head()

,SessionID,item_cnt_per_ses,click_weekday,avg_clicks_items,Time_bucket,top_click_cat,click_hour,session_length,diff_item,item_cnt_in_ses,diff_cat,max_clicks_items_in_ses,pred_target
0,9194351,5,Monday,5,2,S,10,320.0,5,5,1,5,0
1,9194429,2,Monday,2,1,S,6,60.0,1,2,1,2,0
2,9194513,3,Monday,3,2,S,7,883.0,2,3,1,3,0
3,9194692,1,Monday,1,3,S,18,0.0,1,1,1,1,0
4,9194824,1,Monday,1,1,1,1,0.0,1,1,1,1,0


In [42]:
ABT.to_excel("ABT.xlsx")